## Домашняя работа по теме "Классификация. Логистическая регрессия"

1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [1]:
import numpy as np

In [2]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [3]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0, 0.1])
calc_logloss(y1, y_pred1)

<ipython-input-2-7d5907c1794a>:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


inf

In [4]:
def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred == 1, 0.99, y_pred)
    y_pred = np.where(y_pred == 0, 0.01, y_pred)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [5]:
a = np.array([0,1,0.3, 4, 2])

In [6]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0, 0.1])
calc_logloss(y1, y_pred1)

2.3552653508229584

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


In [7]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [8]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [9]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [10]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [11]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [12]:
W = eval_model(X_st, y, iterations=100000, eta=1e-2)

0 [ 0.49292028 -0.15277306  0.64741473  1.51736839] 1.1785958344356262
10000 [-2.51859539 -0.94526548  0.40020792  3.13260413] 0.37535104091786076
20000 [-4.27636089 -0.98163197 -0.27404823  4.26190358] 0.3266452802377501
30000 [-5.56029067 -1.03174275 -0.73817187  5.15128174] 0.29992941321768546
40000 [-6.58505076 -1.08569439 -1.09081936  5.8979985 ] 0.2825314011378889
50000 [-7.45452707 -1.13932291 -1.38011056  6.55229421] 0.2698053402717111
60000 [-8.22283088 -1.19149845 -1.62966903  7.1429085 ] 0.25975500932094964
70000 [-8.9205223  -1.24198713 -1.85228427  7.68713321] 0.2513995524042584
80000 [-9.5660847  -1.29082061 -2.05546494  8.19590938] 0.24420400238940002
90000 [-10.17134369  -1.33810265  -2.24391155   8.67648179] 0.23785183971079435


Минимизировать ошибку получилось с помощью увеличения числа итераций до 100 000 и увеличения скорости обучения до 0.01

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [13]:
def calc_pred_proba(W, X):
    m = X.shape[0]
    y_pred_proba = np.zeros(m)
    A = np.squeeze(sigmoid(np.dot(X, W)))
    for i in range(A.shape[0]):
        y_pred_proba[i] = A[i]
    return y_pred_proba

In [14]:
print(f'Предсказанные вероятности: {calc_pred_proba(W, X_st)}')

Предсказанные вероятности: [0.34909577 0.16500392 0.99720963 0.00128205 0.73111739 0.3237184
 0.99918914 0.09118778 0.40771831 0.99034651]


4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [15]:
def calc_pred(W, X):
    m = X.shape[0]
    y_pred_proba = np.zeros(m)
    A = np.squeeze(sigmoid(np.dot(X, W)))
    for i in range(A.shape[0]):
        if (A[i] > 0.5):
            y_pred_proba[i] = 1
        else:
            y_pred_proba[i] = 0
    return y_pred_proba

In [16]:
y_pred = calc_pred(W, X_st)
print(f'Предсказанные значения {y_pred}')
print(f'Истинные значения {y}')

Предсказанные значения [0. 0. 1. 0. 1. 0. 1. 0. 0. 1.]
Истинные значения [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]


5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [17]:
def accuracy(y, y_pred):
    return np.sum(y == y_pred)/len(y)

print(f'Доля правильных ответов: {accuracy(y, y_pred)}')

Доля правильных ответов: 0.9


In [18]:
import pandas as pd

def matrix(y, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(y.shape[0]):
        if y[i] == y_pred[i] and y[i] == 1:
            TP += 1
        if y[i] == y_pred[i] and y[i] == 0:
            TN += 1
        if not y[i] == y_pred[i]:
            if y[i] == 1:
                FN += 1
        if not y[i] == y_pred[i]:
            if y[i] == 0:
                FP += 1
    
    df = pd.DataFrame(np.array([[f'TP: {TP}', f'FP: {FP}'], [f'FN: {FN}', f'TN: {TN}']]),
                   columns=['+1', '-1'])
    
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f_score = (2*precision*recall)/(precision+recall)
    
    print(f' Матрица ошибок:\n {df}\n***')
    print(f'Точность: {precision}\n***')
    print(f'Полнота: {recall}\n***')
    print(f'Гармоническое среднее: {f_score}')


In [19]:
matrix(y, y_pred)

 Матрица ошибок:
       +1     -1
0  TP: 4  FP: 0
1  FN: 1  TN: 5
***
Точность: 1.0
***
Полнота: 0.8
***
Гармоническое среднее: 0.888888888888889
